In [26]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")
import re
import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [76]:
def data_preprocessing(path, test=False):
    #create appropriate file path
    if test == False:
        pfilename = path + "/product_training.json"
        rfilename = path + "/review_training.json"
    else:
        pfilename = path + "/product_test.json"
        rfilename = path + "/review_test.json"
    
    #extract files as pandas dataframes
    product_df = pd.read_json(pfilename)
    review_df = pd.read_json(rfilename).drop_duplicates(subset=["reviewerID", "unixReviewTime"], keep="first")
    grouped_reviews = review_df.groupby('asin')
    print(grouped_reviews)
    asin_review_data_dict = {}
    #for ind in product_df.index:
        #print(ind)
        #review_group = grouped_reviews.get_group(review_df['asin'][ind])
        #asin_review_data_dict.update({product_df['asin'][ind] : review_group})
        
    return (asin_review_data_dict, product_df)

PSEUDOCODE FOR WHAT WE NEED TO DO WITH THE TRAINING DATA:
 - Saahir/Amy starts by putting all of the reviews for a product in one string. This string is the DOCUMENT for the reviews of a product
 - I will then take the corpus and transform it to remove stopwords, punctuation, and lemmatize everything. This is the true "bag of words".
 - Then, I use the cleaned text data to train the Naive Bayes classifier
 - When we get test data, first clean in the same way, then use it in the classifier

In [7]:
def get_stopwords():
    file = open('en.txt')
    stopwords = []
    for line in file:
        stopwords.append(line.rstrip())
    return stopwords

In [66]:
def transform_document(doc, remove_stopwords = True):
    new_doc = ""
    stopwords = get_stopwords()
    parsed_text = nlp(doc)
    for token in parsed_text:
        lemma = token.lemma_.lower()
        if re.match("[a-z0-9]+", lemma) and (remove_stopwords == False or lemma not in stopwords):
            new_doc += lemma + " "
    return new_doc.rstrip()

transform_document("First, you need to preprocess the raw text data. This may involve tasks like tokenizing the text (i.e., splitting it into individual words), removing stopwords, stemming or lemmatizing the words, and converting the text into a numerical format that can be used as input for the model. Then, you need to split the data into training and testing sets. The training set will be used to train the model, while the testing set will be used to evaluate its performance.")

'preprocess raw text datum involve task tokenize text i.e. split individual word remove stopword stem lemmatize word convert text numerical format input model split datum training testing set training set train model testing set evaluate performance'

In [65]:
def bag_of_words(review_text, remove_stopwords = True):
    word_bag = {}
    stopwords = get_stopwords()
    parsed_text = nlp(review_text)
    for token in parsed_text:
        lemma = token.lemma_.lower()
        if re.match("[a-z0-9]+", lemma) and (remove_stopwords == False or lemma not in stopwords):
            if lemma in word_bag:
                word_bag[lemma] += 1
            else:
                word_bag[lemma] = 1
    return word_bag

In [57]:
def vocabulary_from_corpus(corpus, remove_stopwords = True):
    vocab_set = set()
    for document in corpus:
        word_bag = bag_of_words(document, remove_stopwords)
        for word in word_bag.keys():
            vocab_set.add(word)
    return list(vocab_set)

def transform_document(doc, remove_stopwords = True):
    new_doc = ""
    stopwords = get_stopwords()
    parsed_text = nlp(doc)
    for token in parsed_text:
        lemma = token.lemma_.lower()
        if re.match("[a-z0-9]+", lemma) and (remove_stopwords == False or lemma not in stopwords):
            new_doc += lemma + " "
    return new_doc.rstrip()

vocabulary_from_corpus(['this is the first document', 'this document is the second document', 'and this is the third one', 'is this the first document'], False)

'hand list document document unique word word represent class review'

In [69]:
corpus = ["I am being handed a list of documents", "Each of these documents has several unique words", "The words will represent the class of each review", "I am also removing stopwords in order to make this make more sense"]
cleaned_corpus = [transform_document(doc) for doc in corpus]
vocabulary = vocabulary_from_corpus(cleaned_corpus, True)
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)), ('tfid', TfidfTransformer())]).fit(cleaned_corpus)

['hand list document', 'document unique word', 'word represent class review', 'remove stopword order make make sense']


array([1.91629073, 1.91629073, 1.91629073, 1.51082562, 1.91629073,
       1.91629073, 1.51082562, 1.91629073, 1.91629073, 1.91629073,
       1.91629073, 1.91629073, 1.91629073])

,asin,awesomeness
0,0000B049F5B33CD310EB1AB236E20191,1
1,00018184A9EC4D270219A296B2580303,0
2,000281A9CAC43FF1F335726A390636DA,0
3,00030884DF109F325638A6BFD5B13CFF,1
4,000325BA25966B5FC701D5D2B5DBA4E0,0
...,...,...
73077,FFFDD3C72D23AF858D6E0ED92612370D,1
73078,FFFDDE284A73B29B320381487EC7DE9E,1
73079,FFFEB3EE2372807964F024707D50FB21,0
73080,FFFF4545AB232D81D0F9B208388BB7AA,1


In [77]:
(review_group_dict, product_df) = data_preprocessing("../devided_dataset_v2/CDs_and_Vinyl/train")
#review_group_dict
#review_group_dict['0000B049F5B33CD310EB1AB236E20191']